# Step 1
- Grab the CSV files from https://wellstar-public.conservation.ca.gov/General/PublicDownloads/Index.
 

In [2]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import time

In [7]:
# Define the years we plan to download from CalGEM
years = ['2020','2021','2022','2023']

# Iterate over each year we plan to download.
for year in years:
    print(f'Currently processing year:({year})')

    # Create a dictionary of filenames for each dataset we plan to work with.
    calgem_filenames = {
        "wells_file": f"{year}CaliforniaOilAndGasWells.csv",
        "production_file": f"{year}CaliforniaOilAndGasWellMonthlyProduction.csv",+
        "injection_file": f"{year}CaliforniaOilAndGasWellMonthlyInjection.csv"
    }

    # Construct the CalGEM urls needed to download the datasets.
    wells_url = f"https://calgem-pid.conservation.ca.gov/pid/{calgem_filenames['wells_file']}"
    production_url = f"https://calgem-pid.conservation.ca.gov/pid/{calgem_filenames['production_file']}"
    injections_url = f"https://calgem-pid.conservation.ca.gov/pid/{calgem_filenames['injection_file']}"

    # step to be able to webscrape the data: https://stackoverflow.com/questions/55711159/pandas-read-csv-from-url-and-include-request-header
    # Custom way to tell CalGEM to accept our request for the data.
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11'
    }

    # Create a dataframe for each dataset from CalGEM, and formats the data as CSV.
    df_wells_data = pd.read_csv(wells_url, storage_options=headers)
    df_production_data = pd.read_csv(production_url, storage_options=headers)
    df_injection_data = pd.read_csv(injections_url, storage_options=headers)

    # add column Year to df_wells_data in each year downloaded.
    df_wells_data['ReportYear'] = year
    df_production_data['ReportYear'] = year
    df_injection_data['ReportYear'] = year

    # Writes the downloaded CalGEM data to the CSV file.
    df_wells_data.to_csv(calgem_filenames['wells_file'])
    df_production_data.to_csv(calgem_filenames['production_file'])
    df_injection_data.to_csv(calgem_filenames['injection_file'])

    # Create a connection to the sqlite database.
    disk_engine = create_engine('sqlite:///C://sqlite//capstone.db')

    # Write the data from each dataset into the corresponding database table.
    df_wells_data.to_sql('wells_data', disk_engine, if_exists='append')
    df_production_data.to_sql('production_data', disk_engine, if_exists='append')
    df_injection_data.to_sql('injection_data', disk_engine, if_exists='append')

    # Completed this year. Waiting 5 seconds before we process the next years worth of data.
    print(f'Processing year complete:({year}), waiting 5 sec...')
    time.sleep(5)

# We are done.
print("Download Complete.")

currently processing year:(2020)
processing year complete:(2020), waiting 5 sec...
currently processing year:(2021)
processing year complete:(2021), waiting 5 sec...
currently processing year:(2022)
processing year complete:(2022), waiting 5 sec...
currently processing year:(2023)
processing year complete:(2023), waiting 5 sec...
